In [1]:
##### DNN module

from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, Adagrad, Adam, SGD
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l1, l2, l1_l2

#from sklearn.model_selection import train_test_split
#from keras import backend as K
#from keras.applications import ResNet50

Using TensorFlow backend.


In [2]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
##### Common moldule

import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import time
from PIL import Image
from scipy.interpolate import RegularGridInterpolator
import pandas as pd

In [4]:
##### Control Panel

fn_serial   = '016-A'
batch_size  = 64
num_classes = 5
epochs      = 50
pic_size    = 120
pool_size   = (2, 2)

In [5]:
##### Selection

Conv2D_size = [(3, 3), (5, 5), (7, 7)]
drop_out    = [0., 0.1]
l1_ratio    = [1e-4, 1e-8]
optimizer   = [Adam(lr=0.01),
               Adam(lr=0.001),
               RMSprop(lr=0.01),
               RMSprop(lr=0.001),
               SGD(lr=0.01, momentum=0.95),
               SGD(lr=0.001, momentum=0.95),
              ]

total_item = len(Conv2D_size) * len(drop_out) * len(l1_ratio) * len(optimizer)
print('Total config:', total_item)

Total config: 72


In [6]:
def create_model(model_name, Conv2D_size, pic_size, l1_ratio, pool_size, drop_out_ratio):
    model = Sequential()
    model.name=model_name
    model.add(Conv2D(32, Conv2D_size, padding='same', input_shape=(pic_size, pic_size, 3), kernel_regularizer=l1(l1_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Conv2D(128, Conv2D_size, padding='same', kernel_regularizer=l1(l1_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Conv2D(64, Conv2D_size, padding='same', kernel_regularizer=l1(l1_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Conv2D(32, Conv2D_size, padding='same', kernel_regularizer=l1(l1_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=l1(l1_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(Dropout(drop_out_ratio))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [7]:
def create_compile(optimizer):
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [8]:
def resampleRGI3d(input_mx, resize_to, dtype='float32'):
    # Ref: 10 3-A-7
    # Ver. 2.1
    # input_mx : numpy array, the original target matrix
    # resize_to: list or tuple with 3 int inside
    a, b, c = np.shape(input_mx)
    p, q, r = resize_to
    z_grid = np.linspace(0, p - 1, a)
    y_grid = np.linspace(0, q - 1, b)
    x_grid = np.linspace(0, r - 1, c)
    RGI = RegularGridInterpolator((z_grid, y_grid, x_grid), input_mx)
    z_grid_t2 = np.arange(p)
    y_grid_t2 = np.arange(q)
    x_grid_t2 = np.arange(r)
    meshgrid_para = np.meshgrid(z_grid_t2, y_grid_t2, x_grid_t2)
    RGI_mesh_mx = RGI((meshgrid_para[0], meshgrid_para[1], meshgrid_para[2]))
    RGI_mx = np.transpose(RGI_mesh_mx, axes=[1, 0, 2]).astype(dtype)
    return RGI_mx

def happy_time(start,stop):
    process_time = round(stop - start)
    ss = process_time % 60
    mm = process_time // 60 % 60
    hh = process_time // 3600
    duration = "Process time == {}s == {}H {}m {}s".format(process_time,hh,mm,ss)
    return duration

In [9]:
path_unknown = glob.glob('database/image_data/test/*')

In [10]:
unknown_dict = {}

for j in range(len(path_unknown)):
    serial = os.path.basename(path_unknown[j])[:-4]
    temp_pic = np.asarray(Image.open(path_unknown[j]))
    temp_pic = resampleRGI3d(temp_pic, (pic_size, pic_size, 3))
    temp_shape = np.shape(temp_pic)
    temp_pic = np.expand_dims(temp_pic, axis = 0)
    unknown_dict[serial] = temp_pic

print('Length of dict:', len(list(unknown_dict.keys())))

Length of dict: 2000


In [11]:
##### ImageDataGenerator

augment_generator = ImageDataGenerator(
                                       rotation_range=20,
                                       rescale=1./255,
                                       #featurewise_center=True,
                                       #featurewise_std_normalization=True,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       horizontal_flip=True,
                                       #data_format='channels_last'
                                      )

In [12]:
##### Generator for train and validation

train_generator = augment_generator.flow_from_directory('database/image_data/sep_train',
                                                        target_size=(pic_size, pic_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')
valid_generator = augment_generator.flow_from_directory('database/image_data/sep_valid',
                                                        target_size=(pic_size, pic_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
train_sample = train_generator.samples
valid_sample = valid_generator.samples

Found 2323 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
data batch shape: (64, 120, 120, 3)
labels batch shape: (64, 5)


In [13]:
loop_start = time.time()
count = 0

for i in range(len(Conv2D_size)):
    for j in range(len(drop_out)):
        for k in range(len(l1_ratio)):
            for m in range(len(optimizer)):
                if (m==0) or (m==2):
                    count += 1
                    continue
                if count <= 21:
                    count += 1
                    continue
                model_name = f'Model_{Conv2D_size[i]}-{drop_out[j]}-{l1_ratio[k]}-{optimizer[m]}'
                model = create_model(model_name, Conv2D_size[i], pic_size, l1_ratio[k], pool_size, drop_out[j])
                create_compile(optimizer[m])
                history = model.fit(train_generator, epochs=epochs, verbose=0,
                                    steps_per_epoch=int(train_sample/batch_size),
                                    validation_data=valid_generator,
                                    validation_steps=int(valid_sample/batch_size))
                id_li = []
                flower_class = []
                for n in range(len(list(unknown_dict.keys()))):
                    serial = list(unknown_dict.keys())[n]
                    pred = model.predict(unknown_dict[serial])[0]
                    id_li.append(serial)
                    flower_class.append(list(pred).index(max(pred)))
                    pred_result_df = pd.DataFrame(columns=['id', 'flower_class'])
                    pred_result_df['id'] = id_li
                    pred_result_df['flower_class'] = flower_class
                pred_result_df['flower_class'].value_counts().sort_index()
                pred_result_df.to_csv(f'model/FS_{fn_serial}/pred_result_FS_{i}-{j}-{k}-{m}.csv', index=False)
                train_acc = model.history.history["accuracy"]
                valid_acc = model.history.history["val_accuracy"]
                np.save(f'model/FS_{fn_serial}/History_{i}-{j}-{k}-{m}.npy', [train_acc, valid_acc])
                plt.figure(figsize=(10,8))
                plt.title(f'Accuracy_{i}-{j}-{k}-{m}')
                plt.ylim([0, 1])
                plt.xlim([0, 50])
                plt.plot(train_acc, label = 'train_acc')
                plt.plot(valid_acc, label = 'valid_acc')
                plt.legend()
                plt.savefig(f'model/FS_{fn_serial}/Accuracy_{i}-{j}-{k}-{m}.png')
                plt.clf()
                count += 1
                print(f'{count}/{total_item}, Model_{i}-{j}-{k}-{m} done. {happy_time(loop_start, time.time())}')

23/72, Model_0-1-1-4 done. Process time == 500s == 0H 8m 20s
24/72, Model_0-1-1-5 done. Process time == 1017s == 0H 16m 57s
26/72, Model_1-0-0-1 done. Process time == 1519s == 0H 25m 19s
28/72, Model_1-0-0-3 done. Process time == 2032s == 0H 33m 52s
29/72, Model_1-0-0-4 done. Process time == 2519s == 0H 41m 59s
30/72, Model_1-0-0-5 done. Process time == 3032s == 0H 50m 32s
32/72, Model_1-0-1-1 done. Process time == 3526s == 0H 58m 46s
34/72, Model_1-0-1-3 done. Process time == 4039s == 1H 7m 19s
35/72, Model_1-0-1-4 done. Process time == 4530s == 1H 15m 30s
36/72, Model_1-0-1-5 done. Process time == 5044s == 1H 24m 4s
38/72, Model_1-1-0-1 done. Process time == 5549s == 1H 32m 29s
40/72, Model_1-1-0-3 done. Process time == 6091s == 1H 41m 31s
41/72, Model_1-1-0-4 done. Process time == 6597s == 1H 49m 57s
42/72, Model_1-1-0-5 done. Process time == 7133s == 1H 58m 53s
44/72, Model_1-1-1-1 done. Process time == 7652s == 2H 7m 32s
46/72, Model_1-1-1-3 done. Process time == 8190s == 2H 16m 3

C:\Users\cpjohn\.conda\envs\cpjohn\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


53/72, Model_2-0-0-4 done. Process time == 10800s == 3H 0m 0s
54/72, Model_2-0-0-5 done. Process time == 11329s == 3H 8m 49s
56/72, Model_2-0-1-1 done. Process time == 11829s == 3H 17m 9s
58/72, Model_2-0-1-3 done. Process time == 12366s == 3H 26m 6s
59/72, Model_2-0-1-4 done. Process time == 12868s == 3H 34m 28s
60/72, Model_2-0-1-5 done. Process time == 13402s == 3H 43m 22s
62/72, Model_2-1-0-1 done. Process time == 13924s == 3H 52m 4s
64/72, Model_2-1-0-3 done. Process time == 14464s == 4H 1m 4s
65/72, Model_2-1-0-4 done. Process time == 14989s == 4H 9m 49s
66/72, Model_2-1-0-5 done. Process time == 15549s == 4H 19m 9s
68/72, Model_2-1-1-1 done. Process time == 16069s == 4H 27m 49s
70/72, Model_2-1-1-3 done. Process time == 16620s == 4H 37m 0s
71/72, Model_2-1-1-4 done. Process time == 17143s == 4H 45m 43s
72/72, Model_2-1-1-5 done. Process time == 17698s == 4H 54m 58s


<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

...

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>